In [1]:
import pickle
from pydsge import * # imports eg. the DSGE class and the examples

In [2]:
#Creating a basic pickle example
yaml_file, data_file = example
mod = DSGE.read(yaml_file)
par = mod.set_par('calib')
shock_list = ('e_u', 4.0, 0) # (name, size, period)
X1, (L1, K1), _ = mod.irfs(shock_list, verbose=True)


def is_picklable(obj):
    """
    check if obj can be dumped to a .pkl file
    """
    try:
        pickle.dumps(obj)
    except Exception:
        return False
    return True

bk = {}
for k in dir():
    obj = globals()[k]
    if not k.startswith('_') and is_picklable(obj):
        try:
            bk.update({k: obj})
        except TypeError:
            pass

# to save session
with open('resources/toy_example.pkl', 'wb') as f:
    pickle.dump(bk, f)


[DSGE:]         Parallelization disabled under Windows and Mac due to a problem with pickling some of the symbolic elements. Sorry...


In [1]:
#Restart kernel, then run this

import pickle
from pydsge import * # imports eg. the DSGE class and the examples

In [2]:
# Loading in input
def is_picklable(obj):
    """
    check if obj can be dumped to a .pkl file
    """
    try:
        pickle.dumps(obj)
    except Exception:
        return False
    return True

with open('resources/toy_example.pkl', 'rb') as f:
    bk_restore = pickle.load(f)

In [3]:
#Creating the New version with deliberate difference
yaml_file, data_file = example
mod = DSGE.read(yaml_file)
par = mod.set_par('calib')
shock_list = ('e_u', 3.0, 0) # (name, size, period) ## size 3 instead of 4!!
X1, (L1, K1), _ = mod.irfs(shock_list, verbose=True)

[DSGE:]         Parallelization disabled under Windows and Mac due to a problem with pickling some of the symbolic elements. Sorry...


In [4]:
# Writing test

## Step 1: Get relevant variables in environment
bk_new = {}
for k in dir():
    obj = globals()[k]
    if not k.startswith('_') and is_picklable(obj):
        try:
            bk_new.update({k: obj})
        except TypeError:
            pass

bk_new.keys()

## Step 2: Check whether bk lists are identical [TODO: order of loading will influence the difference, decide for something]
diff = (set(bk_restore.keys()) - set(bk_new.keys())) | (set(bk_new.keys()) - set(bk_restore.keys()))
assert diff == {"bk", "bk_new", "bk_restore"} #[order of loading will influence the difference]

## Step 3: Loop over shared variables
shared_vars = (set(bk_restore.keys()) | set(bk_new.keys())) - diff - {"In"} #TODO: here getting manually rid of "In", but should do that elsewhere

for key in shared_vars:
    if type(bk_new[key]).__name__ == "DataFrame": #Do we really need separate test for DataFrames -> would np.all() not work just as well?
        assert bk_new[key].equals(bk_restore[key]), f"Error with {key}"
    else:
        assert np.all(bk_new[key] == bk_restore[key]), f"Error with {key}" #Use np.all() for arrays

AssertionError: Error with shock_list